<a href="https://colab.research.google.com/github/maskmo/yield-and-transfer-learning/blob/main/Yield_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
from functools import partial

# Transfer Method for CLassification

In [3]:
import tensorflow_datasets as tfds

dataset, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)
dataset_size = info.splits["train"].num_examples
class_names = info.features["label"].names
n_classes = info.features["label"].num_classes

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


In [17]:
test_set_raw, valid_set_raw, train_set_raw = tfds.load(
    "tf_flowers",
    split=["train[:10%]", "train[10%:25%]", "train[25%:]"],
    as_supervised=True)

In [19]:
batch_size = 32
preprocess = tf.keras.Sequential([
    tf.keras.layers.Resizing(height=224, width=224, crop_to_aspect_ratio=True),
    tf.keras.layers.Lambda(tf.keras.applications.xception.preprocess_input)
])
train_set = train_set_raw.map(lambda X, y: (preprocess(X), y))
train_set = train_set.shuffle(1000, seed=42).batch(batch_size).prefetch(1)
valid_set = valid_set_raw.map(lambda X, y: (preprocess(X), y)).batch(batch_size)
test_set = test_set_raw.map(lambda X, y: (preprocess(X), y)).batch(batch_size)

In [6]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip(mode="horizontal", seed=42),
    tf.keras.layers.RandomRotation(factor=0.05, seed=42),
    tf.keras.layers.RandomContrast(factor=0.2, seed=42)
])

In [7]:
base_model = tf.keras.applications.xception.Xception(weights="imagenet",
                                                     include_top=False)
avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(n_classes, activation="softmax")(avg)
model = tf.keras.Model(inputs=base_model.input, outputs=output)

for layer in base_model.layers:
    layer.trainable = False

83683744/83683744 [==============================] - 0s 0us/step


In [8]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=3)

Epoch 1/3
86/86 [==============================] - 29s 173ms/step - loss: 0.7008 - accuracy: 0.8114 - val_loss: 0.6353 - val_accuracy: 0.8385
Epoch 2/3
86/86 [==============================] - 14s 159ms/step - loss: 0.3100 - accuracy: 0.9124 - val_loss: 0.5588 - val_accuracy: 0.8693
Epoch 3/3
86/86 [==============================] - 15s 166ms/step - loss: 0.2125 - accuracy: 0.9335 - val_loss: 0.5285 - val_accuracy: 0.8693


In [10]:
for layer in base_model.layers[56:]:
    layer.trainable = True

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=9)

Epoch 1/9
86/86 [==============================] - 39s 312ms/step - loss: 0.2948 - accuracy: 0.9019 - val_loss: 0.4121 - val_accuracy: 0.8820
Epoch 2/9
86/86 [==============================] - 27s 307ms/step - loss: 0.0417 - accuracy: 0.9876 - val_loss: 0.3423 - val_accuracy: 0.9111
Epoch 3/9
86/86 [==============================] - 28s 311ms/step - loss: 0.0219 - accuracy: 0.9949 - val_loss: 0.3320 - val_accuracy: 0.9074
Epoch 4/9
86/86 [==============================] - 28s 311ms/step - loss: 0.0068 - accuracy: 0.9989 - val_loss: 0.3274 - val_accuracy: 0.9165
Epoch 5/9
86/86 [==============================] - 28s 317ms/step - loss: 0.0033 - accuracy: 0.9993 - val_loss: 0.3366 - val_accuracy: 0.9129
Epoch 6/9
86/86 [==============================] - 29s 328ms/step - loss: 0.0061 - accuracy: 0.9982 - val_loss: 0.3442 - val_accuracy: 0.9038
Epoch 7/9
86/86 [==============================] - 30s 346ms/step - loss: 0.0041 - accuracy: 0.9989 - val_loss: 0.3380 - val_accuracy: 0.9093
Epoch 

# Non Transfer Classification

In [40]:
DefaultConv2D = partial(tf.keras.layers.Conv2D, kernel_size=3, padding="same",
                        activation="relu", kernel_initializer="he_normal")
non_transfer = tf.keras.Sequential([
    DefaultConv2D(filters=64, kernel_size=7, input_shape=[224, 224, 3]),
    tf.keras.layers.MaxPool2D(),
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=64, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=10, activation="softmax")
])

In [47]:
non_transfer.compile(loss="sparse_categorical_crossentropy", optimizer="SGD",
              metrics=["accuracy"])
history = non_transfer.fit(train_set, epochs=9, validation_data=valid_set)
score = non_transfer.evaluate(train_set)
score = model.evaluate(train_set)

Epoch 1/9
86/86 [==============================] - 57s 156ms/step - loss: 1.4191 - accuracy: 0.4204 - val_loss: 1.2638 - val_accuracy: 0.4628
Epoch 2/9
86/86 [==============================] - 14s 149ms/step - loss: 1.3718 - accuracy: 0.4132 - val_loss: 1.2577 - val_accuracy: 0.4646
Epoch 3/9
86/86 [==============================] - 15s 158ms/step - loss: 1.3109 - accuracy: 0.4542 - val_loss: 1.2270 - val_accuracy: 0.4682
Epoch 4/9
86/86 [==============================] - 14s 150ms/step - loss: 1.2727 - accuracy: 0.4720 - val_loss: 1.1360 - val_accuracy: 0.5281
Epoch 5/9
86/86 [==============================] - 15s 161ms/step - loss: 1.2395 - accuracy: 0.4967 - val_loss: 1.1169 - val_accuracy: 0.5172
Epoch 6/9
86/86 [==============================] - 15s 159ms/step - loss: 1.2119 - accuracy: 0.5025 - val_loss: 1.0921 - val_accuracy: 0.5644
Epoch 7/9
86/86 [==============================] - 15s 160ms/step - loss: 1.1487 - accuracy: 0.5356 - val_loss: 1.0825 - val_accuracy: 0.5662
Epoch 

The CNN I used with transfer learning performed with an accuracy of around 0.89 on this data set, while the regular CNN score scores from 0.6-0.7 on each run with different optimizers and hyperparameters. The transfer learning method worked much better.